Logistic Regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000, tolerance=1e-4):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.tolerance = tolerance
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def calculate_gradient(self, X, y, theta):
        m = len(y)
        z = np.dot(X, theta)
        h = self.sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / m
        return gradient
    
    def fit(self, X, y):
        m, n = X.shape
        self.theta = np.zeros(n)  # Initialize weights with zeros
        prev_cost = float('inf')  # ensures that the first iteration of will certainly lead to a decrease in the cost function.
        
        for iteration in range(self.num_iterations):
            gradient = self.calculate_gradient(X, y, self.theta)
            self.theta -= self.learning_rate * gradient
            
            # Calculate the cost function (log-likelihood)
            z = np.dot(X, self.theta)
            h = self.sigmoid(z)
            cost = -(1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
            
            # Check for convergence based on tolerance
            if abs(prev_cost - cost) < self.tolerance:
                print(f"Converged after {iteration + 1} iterations.")
                break
            
            prev_cost = cost
    
    def predict(self, X):
        z = np.dot(X, self.theta)
        h = self.sigmoid(z)
        # Use a threshold of 0.5 for binary classification
        predictions = (h >= 0.5).astype(int)
        return predictions
    
    def predict_proba(self, X):
        z = np.dot(X, self.theta)
        h = self.sigmoid(z)
        return h

In [13]:
import pandas as pd
df = pd.read_csv("./datasets/heartdisease.csv")
categorical = df.select_dtypes(include=['object']).columns.to_list()
numerical = df.select_dtypes(include=['number']).columns.to_list()
print(categorical,'\n', numerical)

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'] 
 ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak', 'HeartDisease']


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


In [23]:
# Get the number of unique values in each column
unique_counts = df.nunique()
# print(unique_counts)
# Filter for columns with less than 10 unique values
columns_with_less_than_10_unique = unique_counts[unique_counts < 10]
encoding_df = df[columns_with_less_than_10_unique.index]
encoding_df

,Sex,ChestPainType,FastingBS,RestingECG,ExerciseAngina,ST_Slope,HeartDisease
0,M,ATA,0,Normal,N,Up,0
1,F,NAP,0,Normal,N,Flat,1
2,M,ATA,0,ST,N,Up,0
3,F,ASY,0,Normal,Y,Flat,1
4,M,NAP,0,Normal,N,Up,0
...,...,...,...,...,...,...,...
913,M,TA,0,Normal,N,Flat,1
914,M,ASY,1,Normal,N,Flat,1
915,M,ASY,0,Normal,Y,Flat,1
916,F,ATA,0,LVH,N,Flat,1


In [ ]:
# Generate some sample data
np.random.seed(0)
X = np.random.randn(100, 2)
y = (X[:, 0] + X[:, 1] > 0).astype(int)  # A simple linear decision boundary
# Split the data into training and test sets (you should use a more robust method in practice)
X_train, X_test = X[:80], X[80:]
y_train, y_test = y[:80], y[80:]

# Create and train the logistic regression model
model = LogisticRegression(learning_rate=0.01, num_iterations=10000, tolerance=1e-4)
model.fit(X_train, y_train)
# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

Metrics

In [ ]:
def calculate_accuracy(y_pred, y_true):
    return np.mean(y_pred == y_true)

def calculate_precision(y_pred, y_true):
    true_positives = np.sum((y_pred == 1) & (y_true == 1))
    false_positives = np.sum((y_pred == 1) & (y_true == 0))
    return true_positives / (true_positives + false_positives)

def calculate_recall(y_pred, y_true):
    true_positives = np.sum((y_pred == 1) & (y_true == 1))
    false_negatives = np.sum((y_pred == 0) & (y_true == 1))
    return true_positives / (true_positives + false_negatives)

def calculate_f1_score(y_pred, y_true):
    precision = calculate_precision(y_pred, y_true)
    recall = calculate_recall(y_pred, y_true)
    return 2 * (precision * recall) / (precision + recall)

def calculate_metric(metric_name, y_pred, y_true):
    if metric_name == "accuracy":
        return calculate_accuracy(y_pred, y_true)
    elif metric_name == "precision":
        return calculate_precision(y_pred, y_true)
    elif metric_name == "recall":
        return calculate_recall(y_pred, y_true)
    elif metric_name == "f1_score":
        return calculate_f1_score(y_pred, y_true)
    elif metric_name == "list_metrics":
        return ["accuracy", "precision", "recall", "f1_score"]
    else:
        raise ValueError(f"Invalid metric name. Available metrics: {', '.join(calculate_metric('list_metrics', None, None))}")

def calculate_metrics(y_pred, y_true):
    metrics = {}
    metrics["accuracy"] = calculate_accuracy(y_pred, y_true)
    metrics["precision"] = calculate_precision(y_pred, y_true)
    metrics["recall"] = calculate_recall(y_pred, y_true)
    metrics["f1_score"] = calculate_f1_score(y_pred, y_true)
    return metrics

def calculate_roc_auc(y_prob, y_true, plot=False):
    thresholds = np.linspace(0, 1, 100)  # Threshold values
    tpr_list = []  # True Positive Rate (Sensitivity)
    fpr_list = []  # False Positive Rate

    for threshold in thresholds:
        y_pred_thresholded = (y_prob >= threshold).astype(int)
        true_positives = np.sum((y_pred_thresholded == 1) & (y_true == 1))
        false_positives = np.sum((y_pred_thresholded == 1) & (y_true == 0))
        true_negatives = np.sum((y_pred_thresholded == 0) & (y_true == 0))
        false_negatives = np.sum((y_pred_thresholded == 0) & (y_true == 1))

        tpr = true_positives / (true_positives + false_negatives)
        fpr = false_positives / (false_positives + true_negatives)

        tpr_list.append(tpr)
        fpr_list.append(fpr)

    auc = calculate_auc(tpr_list, fpr_list)

    if plot:
        plt.figure(figsize=(8, 6))
        plt.plot(fpr_list, tpr_list, linestyle='-', marker='.')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve (AUC: {auc:.2f})')  # Include AUC score in the title
        plt.grid(True)
        # Add AUC score as text annotation on the plot
        plt.annotate(f'AUC = {auc:.2f}', xy=(0.6, 0.4), xytext=(0.6, 0.7),
                    #  arrowprops=dict(facecolor='black', shrink=0.05),
                     fontsize=12, color='black', backgroundcolor='white')
        plt.show()

    return auc

def calculate_auc(tpr, fpr):
    auc = 0.0
    for i in range(1, len(tpr)):
        width = fpr[i] - fpr[i - 1]
        height_avg = (tpr[i] + tpr[i - 1]) / 2
        auc += width * height_avg
    return auc


# Example usage:
if __name__ == "__main__":
    # ... (training and prediction code)

    # Calculate and print the requested metric or list available metrics
    metric_name = "list_metrics"  # Change this to the metric you want to calculate or "list_metrics"
    
    if metric_name == "list_metrics":
        print("Available metrics:", calculate_metric(metric_name, None, None))
    else:
        metric_value = calculate_metric(metric_name, y_pred, y_test)
        print(f"{metric_name.capitalize()}: {metric_value*100:.2f}%")

    # Calculate and print a specific metric
    metric_name = "accuracy"  # Change this to the metric you want
    metric_value = calculate_metric(metric_name, y_pred, y_test)
    print(f"{metric_name.capitalize()}: {metric_value*100:.2f}%")
    
    # Calculate and print all eligible metrics
    calculated_metrics = calculate_metrics(y_pred, y_test)
    for metric_name, metric_value in calculated_metrics.items():
        print(f"{metric_name.capitalize()}: {metric_value*100:.2f}%")

    # Calculate and print ROC AUC score
    auc_score = calculate_roc_auc(y_prob, y_test, plot=True)
    print(f"ROC AUC Score: {auc_score:.2f}")